In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 讀入紅酒資料/了解欄位資訊
* 固定酸度(Fixed Acidity): 與葡萄酒有關的多數酸，為固定或非揮發性（不易蒸發）
* 揮發性酸度(Volatile Acidity): 葡萄酒中乙酸的含量，含量過高會導致令人不快的醋味
* 檸檬酸(Citric Acid): 少量檸檬酸可以增加葡萄酒的“清爽度”和風味
* 殘糖量(Residual Sugar): 
發酵停止後剩餘的糖量，很少發現低於1克/升的葡萄酒，而高於45克/升的葡萄酒被認為是甜的
* 氯化物(Chlorides): 酒中鹽的含量
* 游離二氧化硫(Free Sulfur Dioxide): 存在於分子SO2和亞硫酸氫根離子間平衡的游離形式，可以防止微生物生長和葡萄酒的氧化
* 總二氧化硫(Total Sulfur Dioxide): S02的自由和結合形式的數量
* 密度(Density): 水的密度(根據酒精和糖含量的百分比計算)
* 酸鹼度(pH): 描述葡萄酒的酸性或鹼性程度，從0（非常酸性）到14（非常鹼性）；大多數葡萄酒的pH值在3-4之間
* 硫酸鹽(Sulphates): 一種葡萄酒添加劑，可提高二氧化硫氣體(SO2)水平，起到抗菌和抗氧化劑的作用
* 酒精含量(Alchole): 葡萄酒的酒精含量百分比
* 品質(Quality): 輸出變量（基於感官數據，得分在0到10之間）
* (參考資料: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009)

In [ ]:
data = pd.read_csv('winequality-red.csv')
rows = data.shape[0]
cols = data.shape[1] - 1
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


# 問題轉換為二元分類(品質是否高於平均)

In [ ]:
idxs = (data['quality'] < np.mean(data['quality']))
data.loc[:,'label'] = 1
data.loc[idxs, 'label'] = 0
data.loc[:,['quality', 'label']]

,quality,label
0,5,0
1,5,0
2,5,0
3,6,1
4,5,0
...,...,...
1594,5,0
1595,6,1
1596,6,1
1597,5,0


# 提取出特徵和預測目標

In [ ]:
X = data.iloc[:,:-2]
Y = data.iloc[:,-1]
for i in range(cols):
  X.iloc[:,i] = (X.iloc[:,i] - np.min(X.iloc[:,i])) / (np.max(X.iloc[:,i]) - np.min(X.iloc[:,i]))

# 設定損失函數

In [ ]:
def loss(Y, Y_hat):
  cross_entropy_pos = -1 * (Y * np.log(Y_hat))
  cross_entropy_neg = -1 * ((1 - Y) * np.log(1 - Y_hat))
  cross_entropy_sum = cross_entropy_pos + cross_entropy_neg
  return(np.mean(cross_entropy_sum))

# 設定回歸模型 f(X) = a1x1 + a2x2 + ... + a11x11

In [ ]:
def f(X, a, b):
  Z = np.array([0 for k in range(rows)])
  for i in range(cols):
    Z = Z + X.iloc[:,i] * a[i] + b
  Y = 1 / (1 + np.exp(-Z))
  return(Y)

# 開始隨機搜尋訓練

In [ ]:
a_hat = np.array([0 for k in range(cols)])
b_hat = 0
l_hat = np.Infinity

for i in range(1, 10000):
  a_new = a_hat + np.random.normal(0, 1, cols)
  b_new = b_hat + np.random.normal(0, 1, 1)
  Y_new = f(X, a_new, b_new)
  l_new = loss(Y, Y_new)
  if(l_new < l_hat):
    a_hat = a_new
    b_hat = b_new
    l_hat = l_new
    print('loss = ', l_hat)
  else:
    a_hat = a_hat
    b_hat = b_hat
    l_hat = l_hat

loss =  2.462093882751635
loss =  0.8472966711697634
loss =  0.6854642203793212
loss =  0.6714177245315367
loss =  0.6039866830673407
loss =  0.6025432846418135
loss =  0.6016432785828293
loss =  0.600124686387433
loss =  0.5842828964369193
loss =  0.5748667588903226
loss =  0.5620604538944047
loss =  0.5582411396370026
loss =  0.5507433435939652
loss =  0.5409124892525204
loss =  0.5407219230006567
loss =  0.5334269395373596
loss =  0.5330347050342216
loss =  0.531479280973268
loss =  0.5268843488442161
loss =  0.526696293623214
loss =  0.5228742732207061
loss =  0.5225375004708038


# 計算混淆矩陣

In [ ]:
Y_hat = f(X, a_hat, b_hat)
Y_bin = np.array([0 for k in range(rows)]) 
Y_bin[Y_hat >= 0.5] = 1

TP = sum((Y_bin == 1) & (Y == 1))
TN = sum((Y_bin == 0) & (Y == 0))
FP = sum((Y_bin == 1) & (Y == 0))
FN = sum((Y_bin == 0) & (Y == 1))

accuracy = (TP + TN) / rows
precision = TP / (TP + FP)
recall = TP / (TP + FN)

print('TP =', TP)
print('TN =', TN)
print('FP =', FP)
print('FN =', FN)
print('Accuracy =', np.round(accuracy, 3))
print('Precision =', np.round(precision, 3))
print('Recall =', np.round(recall, 3))

TP = 636
TN = 551
FP = 193
FN = 219
Accuracy = 0.742
Precision = 0.767
Recall = 0.744


# 觀察變數效果並解釋

In [ ]:
pd.DataFrame({'Feature': X.columns, 'Effect': a_hat})

,Feature,Effect
0,fixed acidity,1.699222
1,volatile acidity,-5.578948
2,citric acid,-1.868106
3,residual sugar,-0.493175
4,chlorides,-1.913803
5,free sulfur dioxide,2.561826
6,total sulfur dioxide,-4.489077
7,density,-0.775393
8,pH,-1.495122
9,sulphates,4.618631
